In [1]:
import pandas as pd
import numpy as np
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all


#################
#EveryTime check!
#################

input_data = pd.read_csv('realdata_2017.csv') #データ2

Populating the interactive namespace from numpy and matplotlib


In [2]:
pwd()

'/home/yuichiro/Miniconda/Weather/GribToCSV'

# Set or Modify parameters below!!

In [3]:
input_data[:5]

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Mean Sea Level Pressure [MSL],Radiation [W],Wind Speed [10 m above gnd]
0,2017,1,1,0,0,5.6,1028.4,0.0,1.5
1,2017,1,1,1,0,5.5,1028.7,0.0,1.2
2,2017,1,1,2,0,6.1,1029.1,0.0,0.9
3,2017,1,1,3,0,6.2,1028.8,0.0,1.1
4,2017,1,1,4,0,5.7,1028.2,0.0,1.8


# Solar calc

In [4]:
#太陽光発電初期値

#インプットデータ
#input_rad = pd.read_csv('rad.csv')#encordingはUTF-8なので注意 unitは、MJ/m2
input_csv_parameter = pd.read_csv('PV_input_parameter.csv')

In [5]:
#1列目以外を抽出
input_csv_parameter = input_csv_parameter.iloc[:, [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]]

Gsc = float(input_csv_parameter.iat[0,0])#float(1.367)#the solar constant [1.367 kW/m2]
Gt_NOCT = float(input_csv_parameter.iat[0,1]) #the solar radiation at which the NOCT is defined [0.8 kW/m2]
Ta_NOCT = float(input_csv_parameter.iat[0,2])+273.15 #the ambient temperature at which the NOCT is defined [20°C]
Tc_NOCT = float(input_csv_parameter.iat[0,3])+273.15 #the nominal operating cell temperature [°C]
Tc_STC = float(input_csv_parameter.iat[0,4])+273.15 #the PV cell temperature under standard test conditions [25°C]

YPV = float(input_csv_parameter.iat[0,6]) #the rated capacity of the PV array, meaning its power output under standard test conditions [kW]
ap = float(input_csv_parameter.iat[0,7]) #Module Efficiency(Pmax)[%/℃]
beta= float(input_csv_parameter.iat[0,8])  #the slope of the surface [°]
eta_c = float(input_csv_parameter.iat[0,9]) #the solar absorptance and the solar transmittance[%]
fpv = float(input_csv_parameter.iat[0,10]) #the PV derating factor [%]
ganma = float(input_csv_parameter.iat[0,11]) #the azimuth of the surface [°]
lam=float(input_csv_parameter.iat[0,12])#the longitude [°]
phi=float(input_csv_parameter.iat[0,13])#the latitude [°]
rho_g = float(input_csv_parameter.iat[0,14]) # the ground reflectance, which is also called the albedo [%]
tau_a = float(input_csv_parameter.iat[0,15]) #the solar absorptance and the solar transmittance[%]
_Gt_STC =float(1) #the incident radiation at standard test conditions [1 kW/m2]
UL = (tau_a*Gt_NOCT)/(Tc_NOCT-Ta_NOCT)
#UL = float(input_csv_parameter.iat[0,5])
total_cost = float(input_csv_parameter.iat[0,16]) #total cost
Op_year = float(input_csv_parameter.iat[0,17]) #oparation year
cost = total_cost/(Op_year*365*24)#cost[yen/kW/h]

In [6]:
input_csv_parameter

,Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,YPV,ap,beta,eta_c,fpv,ganma,lam,phi,rho_g,tau_a,total_cost,Op_year
0,1.367,0.8,20,44,25,3,194.44,-0.00258,33,0.194,0.8,0,130.25,33.349,0.2,0.9,199165,15


In [7]:
#入力初期値をデータフレームにする。
input_parameter_name=['Gsc','Gt_NOCT','Ta_NOCT','Tc_NOCT','Tc_STC','UL',\
                      'YPV','ap','beta','eta_c','fpv','ganma','lam','phi',\
                      'rho_g','tau_a','total_cost','Op_year','cost']

input_parameter_list=[Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,\
                      YPV,ap,beta,eta_c,fpv,ganma,lam,phi,\
                      rho_g,tau_a,total_cost,Op_year,cost]

def float2DataFrame(name,list):
    df0=pd.DataFrame({str(name[0]): [list[0]]})
    for i in range(len(list)-1):
        a = float(list[i+1])
        df=pd.DataFrame({str(name[i+1]): [a]})
        df0 =pd.concat([df0, df], axis=1)
    return df0

input_parameter=float2DataFrame(input_parameter_name,input_parameter_list)
input_parameter

,Gsc,Gt_NOCT,Ta_NOCT,Tc_NOCT,Tc_STC,UL,YPV,ap,beta,eta_c,fpv,ganma,lam,phi,rho_g,tau_a,total_cost,Op_year,cost
0,1.367,0.8,293.15,317.15,298.15,0.03,194.44,-0.00258,33.0,0.194,0.8,0.0,130.25,33.349,0.2,0.9,199165.0,15.0,1.515715


In [8]:
input_data.head()

,Year,Month,Day,Hour,Minute,Temperature [2 m above gnd],Mean Sea Level Pressure [MSL],Radiation [W],Wind Speed [10 m above gnd]
0,2017,1,1,0,0,5.6,1028.4,0.0,1.5
1,2017,1,1,1,0,5.5,1028.7,0.0,1.2
2,2017,1,1,2,0,6.1,1029.1,0.0,0.9
3,2017,1,1,3,0,6.2,1028.8,0.0,1.1
4,2017,1,1,4,0,5.7,1028.2,0.0,1.8


# Visualizing

# RealTime validation

In [9]:
#列名の変更
input_data.columns = ['Year', 'Month', 'Day','Hour', 'Minute','Real_Temperature(2m)[°C]', 'Real_Mean Sea Level Pressure[hpa]', \
                          'Real_Radiation[W/m2]','Real_Wind Speed(10m)[m/s]']


#日付型などの変更
input_data['Time'] = input_data[['Year', 'Month', 'Day','Hour']].apply(lambda x: '{}@{}@{}@{}'.format(x[0], x[1], x[2],x[3]), axis=1)
Time = input_data['Time']
input_data['Time'] =pd.to_datetime(Time, format='%Y@%m@%d@%H')

In [10]:
Realtime=pd.concat([input_data],axis=1,sort=False)

In [11]:
#ωの計算
Realtime['ts'] = Realtime['Hour']
Realtime['ω'] = (Realtime['ts'].values - 12 )*15#degree
Realtime['n'] = Realtime['Time'] .dt.dayofyear
Realtime[:5]

,Year,Month,Day,Hour,Minute,Real_Temperature(2m)[°C],Real_Mean Sea Level Pressure[hpa],Real_Radiation[W/m2],Real_Wind Speed(10m)[m/s],Time,ts,ω,n
0,2017,1,1,0,0,5.6,1028.4,0.0,1.5,2017-01-01 00:00:00,0,-180,1
1,2017,1,1,1,0,5.5,1028.7,0.0,1.2,2017-01-01 01:00:00,1,-165,1
2,2017,1,1,2,0,6.1,1029.1,0.0,0.9,2017-01-01 02:00:00,2,-150,1
3,2017,1,1,3,0,6.2,1028.8,0.0,1.1,2017-01-01 03:00:00,3,-135,1
4,2017,1,1,4,0,5.7,1028.2,0.0,1.8,2017-01-01 04:00:00,4,-120,1


In [12]:
Realtime['δ the solar declination']=23.45*np.sin(np.deg2rad(360*(284+Realtime['n'].values)/365))#degree

delta=Realtime['δ the solar declination'].values#degree

In [13]:
#角度をdeg2rad
omega = np.deg2rad(Realtime['ω'].values)#rad
delta =np.deg2rad(delta)#rad
phi =math.radians(phi)#rad
beta =math.radians(beta)#rad
ganma =math.radians(ganma)#rad
lam =math.radians(lam)#rad

#cosθの計算
Realtime['cosθ'] = np.sin(delta)*math.sin(phi)*math.cos(beta)\
                    -np.sin(delta)*math.cos(phi)*math.sin(beta)*math.cos(ganma)\
                    +np.cos(delta)*math.cos(phi)*math.cos(beta)*np.cos(omega)\
                    +np.cos(delta)*math.sin(phi)*math.sin(beta)*math.cos(ganma)*np.cos(omega)\
                    +np.cos(delta)*math.sin(beta)*math.sin(ganma)*np.sin(omega)
                    
cos_theta = Realtime['cosθ'].values

In [14]:
#θの計算
Realtime['θ(radian)'] = np.arccos(Realtime['cosθ'].values)#rad

Realtime['θ(degree)'] = np.rad2deg(Realtime['θ(radian)'].values)#degree

Theta_deg = Realtime['θ(radian)'].values#rad

#cosθzの計算
Realtime['cosθz'] =math.cos(phi)*np.cos(delta)*np.cos(omega)\
                        +math.sin(phi)*np.sin(delta)

cos_theta_z = Realtime['cosθz'].values

Realtime['θz(radian)'] = np.arccos(Realtime['cosθz'].values)#rad

Realtime['θz(degree)'] = np.rad2deg(Realtime['θz(radian)'].values)#degree

Theta_z_deg = Realtime['θz(radian)'].values#rad

#theta degree ndarray
theta = Realtime['θ(radian)'].values#rad

#theta degree ndarray
theta_z = Realtime['θz(radian)'].values#rad

#decimals=1で小数点第2位を四捨五入する
cos_theta = np.round(cos_theta, decimals=1)
cos_theta_z = np.round(cos_theta_z, decimals=1)
Theta_deg =np.round(Theta_deg, decimals=0)
Theta_deg_z = np.round(Theta_z_deg, decimals=0)

cos_theta = np.cos(Theta_deg)
cos_theta_z = np.cos(Theta_z_deg)
Realtime

,Year,Month,Day,Hour,Minute,Real_Temperature(2m)[°C],Real_Mean Sea Level Pressure[hpa],Real_Radiation[W/m2],Real_Wind Speed(10m)[m/s],Time,ts,ω,n,δ the solar declination,cosθ,θ(radian),θ(degree),cosθz,θz(radian),θz(degree)
0,2017,1,1,0,0,5.6,1028.4,0.000,1.5,2017-01-01 00:00:00,0,-180,1,-23.011637,-0.922790,2.746055,157.337363,-0.983768,2.961172,169.662637
1,2017,1,1,1,0,5.5,1028.7,0.000,1.2,2017-01-01 01:00:00,1,-165,1,-23.011637,-0.891427,2.671282,153.053164,-0.957570,2.849244,163.249681
2,2017,1,1,2,0,6.1,1029.1,0.000,0.9,2017-01-01 02:00:00,2,-150,1,-23.011637,-0.799478,2.497222,143.080303,-0.880760,2.648260,151.734145
3,2017,1,1,3,0,6.2,1028.8,0.000,1.1,2017-01-01 03:00:00,3,-135,1,-23.011637,-0.653208,2.282610,130.783923,-0.758573,2.431916,139.338523
4,2017,1,1,4,0,5.7,1028.2,0.000,1.8,2017-01-01 04:00:00,4,-120,1,-23.011637,-0.462585,2.051705,117.554061,-0.599335,2.213467,126.822304
5,2017,1,1,5,0,5.5,1028.5,0.000,1.6,2017-01-01 05:00:00,5,-105,1,-23.011637,-0.240600,1.813781,103.921977,-0.413899,1.997530,114.450028
6,2017,1,1,6,0,5.1,1028.9,0.000,1.8,2017-01-01 06:00:00,6,-90,1,-23.011637,-0.002381,1.573177,90.136430,-0.214902,1.787388,102.409795
7,2017,1,1,7,0,5.4,1029.5,11.112,2.5,2017-01-01 07:00:00,7,-75,1,-23.011637,0.235838,1.332715,76.358970,-0.015905,1.586702,90.911336
8,2017,1,1,8,0,6.3,1029.8,147.234,2.4,2017-01-01 08:00:00,8,-60,1,-23.011637,0.457823,1.095251,62.753278,0.169531,1.400443,80.239467
9,2017,1,1,9,0,9.1,1030.5,336.138,1.5,2017-01-01 09:00:00,9,-45,1,-23.011637,0.648446,0.865255,49.575470,0.328768,1.235798,70.805983


In [15]:
#Gon(the extraterrestrial normal radiation)[kW/m2]の計算
Realtime['Gon(the extraterrestrial normal radiation)[kW/m2]'] \
=Gsc*(1+0.033*np.cos(np.deg2rad(360*Realtime['n'].values/365)))#np.deg2rad

Gon =Realtime['Gon(the extraterrestrial normal radiation)[kW/m2]'].values

#Go(the extraterrestrial horizontal radiation [kW/m2])の計算
Realtime['Go(the extraterrestrial horizontal radiation)[kW/m2]'] \
=Gon*np.abs(cos_theta_z)

Go = Realtime['Go(the extraterrestrial horizontal radiation)[kW/m2]'] .values

In [16]:
#_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]の計算

def step_minus(x):  #再帰関数　引数[2]-引数[1] ndarrayで返す ndarrayを引数
    x = list(x)
    y = [0]
    y.extend(x) #引数2
    x.extend([0]) #引数1
    y = np.array(y)
    x = np.array(x)
    ans = y - x
    return ans[1:]#[1:]or [:8760]


Realtime['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] \
=(12/np.pi)*Gon*np.abs(np.cos(phi)*np.cos(delta)*step_minus(np.sin(omega))\
+((np.pi/180)*step_minus(omega)*np.sin(phi)*np.sin(delta)))

_Go = Realtime['_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]'] 

In [17]:
Realtime['the global horizontal radiation[kW/m2]']=\
Realtime['Real_Radiation[W/m2]'].values/1000

_G = Realtime['the global horizontal radiation[kW/m2]'].values
Realtime['the global horizontal radiation[kW/m2]'] = _G 

In [18]:
#ktの計算
Realtime['kt(the clearness index)'] =_G/_Go #_G2
kt = np.abs(Realtime['kt(the clearness index)'].values)

_Go=kt/_G

#_Gdの計算
#２の３乗#累乗np.power(2, 3)
def if_kt(x):
    x = np.abs(x)
    ans = np.where(x>=0.8,0.165,np.where(x < 0.22, 1-0.09*x,0.95511-0.1604*x+4.388*x*x-16.638*x*x*x+12.336*x*x*x*x))
    return ans

Realtime['_Gd(the diffuse radiation)[kW/m2]']=_G*if_kt(kt)
_Gd = Realtime['_Gd(the diffuse radiation)[kW/m2]'].values

#_Gb(the beam radiation)の計算
Realtime['_Gb(the beam radiation)[kW/m2]']=_G-_Gd

_Gb = Realtime['_Gb(the beam radiation)[kW/m2]'].values

#_G
Realtime['_G']=_Gd+_Gb

/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [19]:
#Rbの計算
#input_data['Rb']= cos_theta/cos_theta_z
Realtime['Rb']= np.abs(Realtime['cosθ'].values/Realtime['cosθz'].values)

#thetaの計算がjupyterだとうまくいかない泣
#input_data['Rb']=Rb
Rb = Realtime['Rb'].values

#発散の処理 要考察 yを超えるとzになる　
def div(x,y,z):
    x = abs(x)
    ans = np.where(x>y,z,x)
    return ans
#Rb = div(Rb,3,3)
#input_data['Rb']= div(Rb,3,3)
Realtime['Rb']= Rb

#Afの計算
Realtime['Af']=_Gb/_Go #_G2

Af = Realtime['Af'].values

#fの計算
Realtime['f']=np.sqrt(_Gb/_G) #_G2
    #欠損値を0で置き換える
Realtime=Realtime.fillna(0)
f = Realtime['f'].values  

/home/yuichiro/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide


In [20]:
def sin3(x):
    return math.sin(x)*math.sin(x)*math.sin(x)

#_Gtの計算#２の３乗#累乗np.power(2, 3) np.power(np.sin(beta/2),3) or def 3sin(x)
_Gt=(_Gb+_Gd*Af)*Rb\
                +(_Gd*(1-Af)*((1+math.cos(beta))/2)\
                *np.abs(1+f*sin3(beta/2)))\
                +(_G*rho_g*((1-math.cos(beta))/2))
_Gt[np.isnan(_Gt)] = 0
        
Realtime['_Gt']  = _Gt                     

In [21]:
#Tc(the PV cell temperature [°C])の計算
Realtime['temperature[K]']=Realtime['Real_Temperature(2m)[°C]']+ 273.15
Ta = Realtime['temperature[K]'].values


#input_data['Tc(the PV cell temperature)[°C]']\
#= Ta+_Gt*(tau_a/UL)*(1-eta_c/tau_a)

Realtime['Tc(the PV cell temperature)[K]']\
= Ta+_Gt*((Tc_NOCT-Ta_NOCT)/Gt_NOCT)*(1-eta_c/tau_a)

Tc = Realtime['Tc(the PV cell temperature)[K]'].values

Realtime['Real_Tc(the PV cell temperature)[°C]']=Realtime['Tc(the PV cell temperature)[K]']-273.15

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用
#input_data['Ppv(the output of the PV array)[Wh]']\
#=div(YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC))), YPV,YPV)

#Ppv(the output of the PV array)[kW]の計算   #発散の処理div(x)を使用2
Realtime['Real_Ppv(the output of the PV array)[Wh]']\
=1.28296*YPV*fpv*(_Gt/_Gt_STC)*np.abs((1+ap*(Tc-Tc_STC)))

Ppv = Realtime['Real_Ppv(the output of the PV array)[Wh]'].values

#input_data['Left'] = YPV*fpv*(_Gt/_Gt_STC)
#input_data['Right'] =np.abs((1+ap*(Tc-Tc_STC)))

Realtime[:10]  

,Year,Month,Day,Hour,Minute,Real_Temperature(2m)[°C],Real_Mean Sea Level Pressure[hpa],Real_Radiation[W/m2],Real_Wind Speed(10m)[m/s],Time,...,_Gb(the beam radiation)[kW/m2],_G,Rb,Af,f,_Gt,temperature[K],Tc(the PV cell temperature)[K],Real_Tc(the PV cell temperature)[°C],Real_Ppv(the output of the PV array)[Wh]
0,2017,1,1,0,0,5.6,1028.4,0.000,1.5,2017-01-01 00:00:00,...,0.000000,0.000000,0.938015,0.000000,0.000000,0.000000,278.75,278.750000,5.600000,0.000000
1,2017,1,1,1,0,5.5,1028.7,0.000,1.2,2017-01-01 01:00:00,...,0.000000,0.000000,0.930927,0.000000,0.000000,0.000000,278.65,278.650000,5.500000,0.000000
2,2017,1,1,2,0,6.1,1029.1,0.000,0.9,2017-01-01 02:00:00,...,0.000000,0.000000,0.907714,0.000000,0.000000,0.000000,279.25,279.250000,6.100000,0.000000
3,2017,1,1,3,0,6.2,1028.8,0.000,1.1,2017-01-01 03:00:00,...,0.000000,0.000000,0.861102,0.000000,0.000000,0.000000,279.35,279.350000,6.200000,0.000000
4,2017,1,1,4,0,5.7,1028.2,0.000,1.8,2017-01-01 04:00:00,...,0.000000,0.000000,0.771831,0.000000,0.000000,0.000000,278.85,278.850000,5.700000,0.000000
5,2017,1,1,5,0,5.5,1028.5,0.000,1.6,2017-01-01 05:00:00,...,0.000000,0.000000,0.581302,0.000000,0.000000,0.000000,278.65,278.650000,5.500000,0.000000
6,2017,1,1,6,0,5.1,1028.9,0.000,1.8,2017-01-01 06:00:00,...,0.000000,0.000000,0.011080,0.000000,0.000000,0.000000,278.25,278.250000,5.100000,0.000000
7,2017,1,1,7,0,5.4,1029.5,11.112,2.5,2017-01-01 07:00:00,...,0.000027,0.011112,14.827789,0.000011,0.049449,0.010786,278.55,278.803832,5.653832,2.259986
8,2017,1,1,8,0,6.3,1029.8,147.234,2.4,2017-01-01 08:00:00,...,0.002469,0.147234,2.700532,0.001614,0.129502,0.142941,279.45,282.813887,9.663887,29.655089
9,2017,1,1,9,0,9.1,1030.5,336.138,1.5,2017-01-01 09:00:00,...,0.049399,0.336138,1.972351,0.042167,0.383354,0.381414,282.25,291.225940,18.075940,77.477395


In [22]:
rf = Realtime.set_index('Time')
rf.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'Real_Temperature(2m)[°C]',
       'Real_Mean Sea Level Pressure[hpa]', 'Real_Radiation[W/m2]',
       'Real_Wind Speed(10m)[m/s]', 'ts', 'ω', 'n', 'δ the solar declination',
       'cosθ', 'θ(radian)', 'θ(degree)', 'cosθz', 'θz(radian)', 'θz(degree)',
       'Gon(the extraterrestrial normal radiation)[kW/m2]',
       'Go(the extraterrestrial horizontal radiation)[kW/m2]',
       '_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]',
       'the global horizontal radiation[kW/m2]', 'kt(the clearness index)',
       '_Gd(the diffuse radiation)[kW/m2]', '_Gb(the beam radiation)[kW/m2]',
       '_G', 'Rb', 'Af', 'f', '_Gt', 'temperature[K]',
       'Tc(the PV cell temperature)[K]',
       'Real_Tc(the PV cell temperature)[°C]',
       'Real_Ppv(the output of the PV array)[Wh]'],
      dtype='object')

In [23]:
file_dir_path = "Output_CSV"

In [24]:
#データの書き出し
#filename = file_dir_path+'/realtime'+year+month+day+time+'.csv'
#Realtime.to_csv(filename,encoding="UTF-8")  



# Wind calc

In [25]:
popt = np.array([ 0.00220261, -0.00783915,  0.00496154])

def power_curve_func(x,cutin,popt):
    return np.where(x > 14, 0.3,np.where(x < cutin, 0, popt[0]*x**2+popt[1]*x+popt[2]))

def wind_turbine_output(wind_values,temp_values,MSL_values,Pr,V_rat, V_CIN, V_CO):   
    # Pr:rated power, V_rat: rated wind speed,V_CIN: cut-in speed, V=CO: cut-out speed,rho: air dencity
    rho0 = 1.225
    rho = MSL_values / (2.87*273.15+2.87*temp_values)
    Pwt=(np.where((V_rat<=wind_values) & (wind_values <= V_CO), Pr,\
                  np.where((V_CIN<=wind_values) & (wind_values <= V_rat), \
                           Pr*(wind_values-V_CIN)/(V_rat-V_CIN),0 )))
    return (Pwt*(rho/rho0))

In [26]:
# Realtime['Mean Sea Level Pressure[hpa]'] Realtime['Temperature(2m)[°C]']
#Realtime['Wind Speed(10m)[m/s]']

# def wind_turbine_output(wind_values,temp_values,MSL_values,Pr,V_rat, V_CIN, V_CO)
Pwt = wind_turbine_output(Realtime['Real_Wind Speed(10m)[m/s]'].values, Realtime['Real_Temperature(2m)[°C]'].values,
                          Realtime['Real_Mean Sea Level Pressure[hpa]'].values, 0.3, 20, 3, 100)
print(Pwt.shape)
# def power_curve_func(x,cutin,popt)
Pwt_from_power_curve = power_curve_func(
    Realtime['Real_Wind Speed(10m)[m/s]'].values, 3, popt)

Realtime['Pwt[kW]'] = pd.Series(Pwt)
Realtime['Pwt from power curve[kW]'] = pd.Series(Pwt_from_power_curve)

(8760,)


# Visualize realtime

In [27]:
df_real = Realtime.set_index('Time')
df_real.columns

Index(['Year', 'Month', 'Day', 'Hour', 'Minute', 'Real_Temperature(2m)[°C]',
       'Real_Mean Sea Level Pressure[hpa]', 'Real_Radiation[W/m2]',
       'Real_Wind Speed(10m)[m/s]', 'ts', 'ω', 'n', 'δ the solar declination',
       'cosθ', 'θ(radian)', 'θ(degree)', 'cosθz', 'θz(radian)', 'θz(degree)',
       'Gon(the extraterrestrial normal radiation)[kW/m2]',
       'Go(the extraterrestrial horizontal radiation)[kW/m2]',
       '_Go(the extraterrestrial horizontal radiation averaged over the time step )[kW/m2]',
       'the global horizontal radiation[kW/m2]', 'kt(the clearness index)',
       '_Gd(the diffuse radiation)[kW/m2]', '_Gb(the beam radiation)[kW/m2]',
       '_G', 'Rb', 'Af', 'f', '_Gt', 'temperature[K]',
       'Tc(the PV cell temperature)[K]',
       'Real_Tc(the PV cell temperature)[°C]',
       'Real_Ppv(the output of the PV array)[Wh]', 'Pwt[kW]',
       'Pwt from power curve[kW]'],
      dtype='object')

In [28]:
#データの書き出し
filename = file_dir_path+'/realtime_result.csv'
Realtime.to_csv(filename,encoding="SHIFT-JIS")  
